# Quantum Approximate Optimization Algorithm

In this assignment you will implement Quantum Approximate Optimization Algorithm (QAOA) to find bit strings that consist of alternating 0s and 1s.

In [1]:
import qsharp
import scipy.optimize as sco


In [2]:
%%qsharp

open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;


### Task 1. Classical cost function (1 point)

The cost function $C(x)$ defines the cost of each possible input, and should be minimized for the inputs we're looking for - in this case the bit strings with alternating 0s and 1s $0101...$ and $1010...$.
This cost function can be defined as the number of pairs of identical bits in adjacent positions:

$$C(x) = \sum_{i=0}^{N - 2} x_i = x_{i+1}$$

For bit strings with alternating bits, all terms of the sum will be 0, achieving the minimal possible value of this cost function.

**Input**: an array of integers $x$, each of them 0 or 1.

**Output**: the value of the cost function $C(x)$.

In [3]:
# Implement the cost function C 
def cost(x):
    count =0
    for i in range(0,len(x)-1):
        j=i+1
        if (x[i]==x[j]):
            count += 1
        else:
            count +=0
    return count

# Evaluate the function on several inputs to check it's correct
for x in [[1, 0, 1],
          [0, 1, 0, 1],
          [1, 1, 1, 1],
          [0, 1, 1, 0]]:
    print(f"C({x}) = {cost(x)}")

C([1, 0, 1]) = 0
C([0, 1, 0, 1]) = 0
C([1, 1, 1, 1]) = 3
C([0, 1, 1, 0]) = 1


### Task 2. QAOA Phase Change Unitary (4 points)

**Inputs:**
1. A qubit array of type `Qubit[]` - the input of the phase oracle.
2. A `Double` - the value of the parameter $\gamma$.

**Goal:** 
Implement the first unitary used in QAOA - the phase change unitary. This unitary should implement the transformation

$$U_C(\gamma)|\psi\rangle = e^{-i\gamma C(x)}|\psi\rangle$$

> This cost function can be expressed as a sum of terms applied to adjacent pairs of qubits. This allows you to decompose the $U_C$ operator into a sequence of 1- and 2-qubit operations.

In [4]:
%%qsharp
operation AlternatingBitStringQAOAOracle(qs : Qubit[], gamma : Double) : Unit is Adj + Ctl {
    // ...
    for i in 0 .. Length(qs) - 2 {
        ApplyControlledOnBitString([false,false], Rz, [qs[i], qs[i+1]], (-2.0 * gamma, qs[i]));
        ApplyControlledOnBitString([true,true], Rz, [qs[i], qs[i+1]], (-2.0 * gamma, qs[i]));
    }
}

The code below can be used to test your implementation. The `AlternatingBitStringQAOAOracleDemo` operation applies the `AlternatingBitStringQAOAOracle` operation to $N$ qubits, each in the $|+\rangle$ state. Here are a couple of useful examples for testing:

* For $N = 2$, the cost function $C(x)$ can take values 0 or 1. $\gamma = \pi$ allows to separate these cases nicely: basis states with $C(x) = 1$ will get $-1$ relative phase.
* For $N = 3$, the cost function $C(x)$ can take values 0, 1 or 2. This means that with $\gamma = \frac{\pi}{2}$ a basis state will remain unchanged ($C=0$) or acquire a relative phase $e^{-i\frac{\pi}{2}} = -i$ ($C=1$) or $e^{-i\pi} = -1$ ($C=2$).

Use the output of `DumpMachine` to check whether each basis state gains the correct phases. Remember that `DumpMachine` output can have an additional global phase.

In [5]:
%%qsharp
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation AlternatingBitStringQAOAOracleDemo(N : Int, gamma : Double) : Unit {
    use qs = Qubit[N];
    ApplyToEach(H, qs);
    AlternatingBitStringQAOAOracle(qs, gamma);
    DumpMachine();
    ResetAll(qs);
}

AlternatingBitStringQAOAOracleDemo(2, PI())
AlternatingBitStringQAOAOracleDemo(3, PI()/2.0)

STATE:
|00⟩: −0.5000+0.0000𝑖
|01⟩: 0.5000+0.0000𝑖
|10⟩: 0.5000+0.0000𝑖
|11⟩: −0.5000+0.0000𝑖

STATE:
|000⟩: −0.3536+0.0000𝑖
|001⟩: 0.0000−0.3536𝑖
|010⟩: 0.3536+0.0000𝑖
|011⟩: 0.0000−0.3536𝑖
|100⟩: 0.0000−0.3536𝑖
|101⟩: 0.3536+0.0000𝑖
|110⟩: 0.0000−0.3536𝑖
|111⟩: −0.3536+0.0000𝑖

### Task 3. QAOA Mixer Unitary (3 points)

**Inputs:**
1. A qubit array of type `Qubit[]` - the input of the mixer unitary.
2. A `Double` - the value of the parameter $\beta$.

**Goal:** 
Implement the second unitary used in QAOA - the mixer. This unitary should implement the transformation
$${\displaystyle \prod_{j=0}^{N-1} e^{-i\beta X_j}},$$
where $X_j$ is the Pauli X operator acting on qubit j. By definition,
$$e^{i\theta X} = \begin{bmatrix} \cos(\theta) & i\sin(\theta) \\ i\sin(\theta) & cos(\theta) \end{bmatrix}$$

In [6]:
%%qsharp
operation Mixer(qs : Qubit[], beta : Double) : Unit is Adj + Ctl {
    // ...
    let N= Length(qs);
    for i in 1 .. N {
        Rx(beta * 2.0, qs[i-1]);
    }
}

The code below can be used to test your implementation. It will output the matrix representation of the `Mixer` operation called with 1 qubit and several different parameters $\beta$. 

> Remember that, same as `DumpMachine`, output of `dump_operation` can have a global phase; if you need to get rid of it, modify the Q# code to add a wrapper that calls a controlled variant of `Mixer` (the wrapper should not have specializations), and use `dump_operation` to print the matrix of that controlled variant.

In [7]:
from qsharp.utils import dump_operation

print(dump_operation(f"Mixer(_, 0.0)", 1))
print(dump_operation(f"Mixer(_, PI() / 4.0)", 1))
print(dump_operation(f"Mixer(_, PI() / 2.0)", 1))

[[(1+0j), 0j], [0j, (1+0j)]]
[[(0.707107+0j), -0.707107j], [-0.707107j, (0.707107+0j)]]
[[0j, (1+0j)], [(1+0j), 0j]]


### Task 4. Implement depth 1 QAOA to find the optimal parameters $\beta$ and $\gamma$ (2 points)

**Inputs:**

1. $N$ - the length of the bit string in the optimization problem.
2. `gamma` and `beta` - `Double` values $\gamma$ and $\beta$.

**Output:**

An array of type `Int` representing measurement results of the $N$ qubits used in QAOA: 0 and 1 values represent measurement results `Zero` and `One`, respectively.

**Goal:** Implement operation `RunQAOA` to run depth 1 QAOA with the given parameters $\beta$ and $\gamma$. Then, use the Python code below to perform classical optimization to identify optimal $\gamma$ and $\beta$ values for the circuit, and use these parameters to get the solution to the problem for N = 2 .. 5.

In [8]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;

operation RunQAOA(N : Int, gamma : Double, beta : Double) : Int[] {
    // ...
    mutable result = [];
    use qubits = Qubit[N];
    ApplyToEach(H, qubits);
    AlternatingBitStringQAOAOracle(qubits, gamma);
    Mixer(qubits, beta);
    for i in qubits {
        if (M(i) == Zero) {
            set result = result + [0];
        } else {
            set result = result + [1];
        }
    }
    ResetAll(qubits);
    return result;
}

The following Python code uses classical optimization method from `scipy.optimize` to find the parameters $\beta$ and $\gamma$ that minimize the average cost of the configuration produced by the QAOA run for bit strings of different lengths. Note that if your solution is correct, you should get correct results (bit strings with alternating bits) with reasonably high probability.

In [9]:
from functools import partial

def multishot_cost(N, parameters):
    shots = 30
    total_cost = 0
    for i in range(shots):
        result = qsharp.eval(f"RunQAOA({N}, {parameters[0]}, {parameters[1]})")
        total_cost += cost(result) 
    return total_cost / shots

def answer_freq(N, parameters):
    dict = {}
    shots = 100
    for i in range(shots):
        result = qsharp.eval(f"RunQAOA({N}, {parameters[0]}, {parameters[1]})")
        key = ''.join(map(str, result))
        if key in dict:
            dict[key] = dict[key] + 1
        else:
            dict[key] = 1
    return {k : (v / shots) for k, v in dict.items()}

for N in range(2, 6):
    out = sco.minimize(partial(multishot_cost, N), [1, 1], method="COBYLA", options={'maxiter':50})
    print(f"Optimal Values for N = {N}:")
    print("γ = {:.2f}".format(out.x[0]))
    print("β = {:.2f}".format(out.x[1]))
    ans = answer_freq(N, out.x)
    print(sorted(ans.items(), key=lambda x:x[1], reverse=True))

Optimal Values for N = 2:
γ = 0.98
β = 1.23
[('10', 0.45), ('01', 0.43), ('00', 0.07), ('11', 0.05)]
Optimal Values for N = 3:
γ = 0.87
β = 1.13
[('101', 0.36), ('010', 0.34), ('100', 0.09), ('110', 0.07), ('011', 0.06), ('001', 0.05), ('111', 0.03)]
Optimal Values for N = 4:
γ = 1.15
β = 1.14
[('1010', 0.27), ('0101', 0.24), ('1001', 0.15), ('0110', 0.13), ('0010', 0.06), ('1011', 0.04), ('0100', 0.03), ('1111', 0.02), ('1100', 0.02), ('1101', 0.02), ('0000', 0.01), ('0011', 0.01)]
Optimal Values for N = 5:
γ = 1.01
β = 1.19
[('10101', 0.25), ('01010', 0.14), ('01001', 0.1), ('10110', 0.09), ('10010', 0.08), ('01011', 0.06), ('00101', 0.05), ('01101', 0.05), ('01100', 0.05), ('10100', 0.03), ('01111', 0.03), ('10011', 0.02), ('00110', 0.02), ('11110', 0.01), ('11010', 0.01), ('11011', 0.01)]
